In [1]:
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 239 (delta 86), reused 53 (delta 53), pack-reused 121 (from 2)
Receiving objects: 100% (239/239), 2.18 MiB | 13.44 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [2]:
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

In [3]:
# Load the model
VARIANT = "2b-v2" 
MACHINE_TYPE = "cuda"
weights_dir = '/kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1'
weights_file = os.path.join(weights_dir, "model.ckpt")

In [4]:
!pip install gputil psutil matplotlib

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=8e7b2073e133a311a54d66abc59af57f58dac1dfbb772b0f0e6027dd126a9460
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [5]:
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

In [6]:
import threading
import time
import psutil  # To monitor CPU, RAM, GPU usage
import GPUtil  # To monitor GPU usage

class SystemMonitor:
    def __init__(self, interval=1):
        # Variables to store usage and timestamps
        self.gpu_loads = []  # GPU usage in percentage
        self.gpu_memories_used = []  # GPU memory in GB
        self.cpu_usages = []  # CPU usage in percentage
        self.ram_usages_used = []  # RAM usage in GB
        self.time_stamps = []  # Timestamps
        self.interval = interval  # Interval in seconds
        self.monitoring = False
        self.monitor_thread = None

    def _monitor_system(self):
        """
        Monitors system usage (CPU, GPU, RAM) and records the data at the given interval.
        """
        # Capture current time
        start_time = time.time()
        while self.monitoring:
            
            # Record the current timestamp
            timestamp = time.time() - start_time
            self.time_stamps.append(timestamp)
            
            # Get CPU usage
            self.cpu_usages.append(psutil.cpu_percent(interval=None))
            
            # Get RAM usage
            ram = psutil.virtual_memory()
            self.ram_usages_used.append(ram.used / (1024 ** 3))  # Convert bytes to GB

            # Get GPU usage
            gpus = GPUtil.getGPUs()
            if gpus:
                gpu = gpus[0]  # Assuming you're monitoring the first GPU
                self.gpu_loads.append(gpu.load * 100)  # Convert to percentage
                self.gpu_memories_used.append(gpu.memoryUsed / 1024)  # Convert MB to GB
                
            # Sleep for the interval before next measurement
            time.sleep(self.interval)

    def start_monitoring(self):
        """Starts the system monitoring in a separate thread."""
        if not self.monitoring:
            self.monitoring = True
            self.monitor_thread = threading.Thread(target=self._monitor_system)
            self.monitor_thread.start()
            print("Monitoring started.")

    def stop_monitoring(self):
        """Stops the system monitoring."""
        if self.monitoring:
            self.monitoring = False
            self.monitor_thread.join()
            print("Monitoring stopped.")

    def get_data(self):
        """Returns the collected monitoring data."""
        return {
            "time_stamps": self.time_stamps,
            "gpu_loads": self.gpu_loads,
            "gpu_memories_used": self.gpu_memories_used,
            "cpu_usages": self.cpu_usages,
            "ram_usages_used": self.ram_usages_used
        }


In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statistics import mean

def plot_system_usage(data):
    """
    Plots system resource usage (GPU, CPU, RAM) over time using Plotly.
    
    Args:
        data (dict): Dictionary containing system monitoring data with keys:
                     'time_stamps', 'gpu_loads', 'gpu_memories_used', 'cpu_usages', 'ram_usages_used'.
    """
    # Create subplots
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=('GPU Usage (%)', 'GPU Memory Usage (GB)', 'CPU Usage (%)', 'RAM Usage (GB)'),
        shared_xaxes=True,
        vertical_spacing=0.05
    )

    # Plot GPU usage
    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y = data['gpu_loads'], mode='lines', name='GPU Usage (%)', line=dict(color='blue')),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y = [mean(data['gpu_loads'])] * len(data['time_stamps']), mode='lines', 
                   name='GPU Avg (%)', line=dict(color='blue', dash='dash')),
        row=1, col=1
    )

    # Plot GPU memory usage
    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y=data['gpu_memories_used'], mode='lines', name='GPU Memory Usage (GB)', line=dict(color='purple')),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y = [mean(data['gpu_memories_used'])] * len(data['time_stamps']), mode='lines', 
                   name='GPU Memory Avg (GB)', line=dict(color='purple', dash='dash')),
        row=2, col=1
    )

    # Plot CPU usage
    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y=data['cpu_usages'], mode='lines', name='CPU Usage (%)', line=dict(color='orange')),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y = [mean(data['cpu_usages'])] * len(data['time_stamps']), mode='lines', 
                   name='CPU Avg (%)', line=dict(color='orange', dash='dash')),
        row=3, col=1
    )

    # Plot RAM usage
    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y=data['ram_usages_used'], mode='lines', name='RAM Usage (GB)', line=dict(color='green')),
        row=4, col=1
    )

    fig.add_trace(
        go.Scatter(x=data['time_stamps'], y = [mean(data['ram_usages_used'])] * len(data['time_stamps']), mode='lines', 
                   name='Ram Usage Avg (GB)', line=dict(color='green', dash='dash')),
        row=4, col=1
    )


    # Update layout with titles, axis labels, and grid
    fig.update_layout(
        height=800, width=800,
        title_text="System Resource Usage Over Time",
        xaxis=dict(tickmode='linear'),
        xaxis2=dict(showgrid=True), 
        xaxis3=dict(showgrid=True),
        xaxis4=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        yaxis2=dict(showgrid=True),
        yaxis3=dict(showgrid=True),
        yaxis4=dict(showgrid=True),
    )
    fig.update_yaxes(title_text="Percentage", row=1, col=1)
    fig.update_yaxes(title_text="GB", row=2, col=1)
    fig.update_yaxes(title_text="Percentage", row=3, col=1)
    fig.update_yaxes(title_text="GB", row=4, col=1)
    fig.update_xaxes(title_text="Time (seconds)", row=4, col=1)

    # Show the plot
    fig.show()

## Loading the model to GPU

In [8]:
model_config = get_model_config(VARIANT)
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")

In [9]:
sys_mon = SystemMonitor()
sys_mon.start_monitoring()

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  model.load_weights(weights_file)
  model = model.to(device).eval()

sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [23]:
plot_system_usage(sys_mon.get_data())

In [24]:
# Use the model
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn><eos>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn><eos>\n"

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="What is a good place for travel in the US?"
    )
    + MODEL_CHAT_TEMPLATE.format(prompt="California.")
    + USER_CHAT_TEMPLATE.format(prompt="What can I do in California?")
    + "<start_of_turn>model\n"
)

In [12]:
sys_mon = SystemMonitor()
sys_mon.start_monitoring()

response = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=500,
)

# Stop monitoring

sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [13]:
print(response)

California is big and diverse! To help me suggest something, tell me a little more about what you enjoy:

* **What are your interests?** (Nature, cities, beaches, history, food, nightlife?)
* **Who are you travelling with?** (Solo, couple, family, friends?)
* **How much time do you have?** (Weekend trip, week long vacation?)  
* **What's your budget like?** (Luxury, budget-friendly, somewhere in between?)

Once you tell me a bit more, I can recommend some amazing California experiences! 🌴🌊⛰️🥂 
<end_of_turn>


In [14]:
plot_system_usage(sys_mon.get_data())

In [25]:
input_data = "What is the capital of France?"

sys_mon = SystemMonitor()
sys_mon.start_monitoring()

response = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=input_data)
    + "<start_of_turn>model\n",
    device=device,
    output_len=500,
)

# Stop monitoring
sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [26]:
print(response)

The capital of France is **Paris**. 🇫🇷  
<end_of_turn>


In [27]:
plot_system_usage(sys_mon.get_data())

In [28]:
input_data = "म पुस्तक पढिरहेको छु। Translate to English."

sys_mon = SystemMonitor()
sys_mon.start_monitoring()

response = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=input_data)
    + "<start_of_turn>model\n",
    device=device,
    output_len=500,
)

# Stop monitoring
sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [19]:
print(response)

I am reading a book. 
<end_of_turn>


In [29]:
plot_system_usage(sys_mon.get_data())

### Calculating average inference time

In [31]:
input_data = "नेपालको राजधानी के हो?"

sys_mon = SystemMonitor()
sys_mon.start_monitoring()

num_repeats = 10
for _ in range(num_repeats):
    response = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=input_data)
        + "<start_of_turn>model\n",
        device=device,
        output_len=500,
    )
    print(response)

# Stop monitoring
sys_mon.stop_monitoring()

Monitoring started.
नेपालको राजधानी **Kathmandu** छ. 
<end_of_turn>
नेपालको राजधानी **पखडुरा** हो। 
<end_of_turn>
नेपालको राजधानी **पेशवा**  (also known as Kathmandu)  हुन्छ । 
<end_of_turn>
नेपालको राजधानी **पटना** हो । 
<end_of_turn>
नेपालको राजधानी **Kathmandu** छ। 
<end_of_turn>
नेपालको राजधानी **पेरिफाँन** हो। 
<end_of_turn>
नेपालको राजधानी **पताथी** हो। 
<end_of_turn>
नेपालको राजधानी **जम्मू दर्रा**  हुन्छ । 
<end_of_turn>
नेपालको राजधानी **पखान्से** हो । 
<end_of_turn>
नेपालको राजधानी **केन्द्रीय** मुख्यालय **पखड** हो। 
<end_of_turn>
Monitoring stopped.


In [32]:
plot_system_usage(sys_mon.get_data())

- 10 inferences completed in 228.2402
- 1 inference = 22.82402 in average
- output length = 500

## Summarization Test

In [33]:
input_data = """
नेपाल दक्षिण एशियाको एक सानो, तर विविधताले भरिएको देश हो। यो देश विश्वका १४ अग्ला हिमालमध्ये ८ वटा हिमालको घर हो, जसमा सगरमाथा (माउन्ट एभरेस्ट) समेत समावेश छ। नेपालको प्राकृतिक सौन्दर्यले मात्र होइन, सांस्कृतिक र ऐतिहासिक महत्त्वले पनि यसलाई विशेष बनाएको छ। यहाँ विभिन्न जातजाति, भाषा, धर्म र संस्कृति समेटिएका छन्। नेपालमा नेवार, मगर, थारु, लिम्बु, शेर्पाजस्ता जातिहरूको बसोबास छ, जसले नेपाली समाजलाई विविधताले भरिपूर्ण बनाएको छ।

धार्मिक दृष्टिकोणले, नेपाल हिन्दू र बौद्ध धर्मका लागि एक पवित्र स्थल मानिन्छ। लुम्बिनी बुद्धको जन्मस्थल हो, जहाँ विश्वभरका पर्यटकहरू दर्शनका लागि आउने गर्छन्। पशुपतिनाथ मन्दिर, चाङ्गुनारायण, स्वयम्भूनाथ, बौद्धनाथ जस्ता धार्मिक स्थलहरू पनि नेपालका मुख्य आकर्षण हुन्। नेपालको राजधानी काठमाडौं तीन हजार वर्ष पुरानो इतिहास बोकेको शहर हो। यहाँ मल्ल राजाहरूले निर्माण गरेका दरबार, मन्दिर र कलात्मक सम्पदा भेट्न सकिन्छ।

नेपालको प्राकृतिक सौन्दर्यले साहसिक पर्यटकहरूलाई आकर्षित गर्छ। अन्नपूर्ण बेस क्याम्प, लाङटाङ उपत्यका, रारा ताल, फेवा तालजस्ता गन्तव्यहरूमा हाइकिङ, ट्रेकिङ, र क्याम्पिङ गर्न पर्यटकहरू आउने गर्छन्। वन्यजन्तु संरक्षणका लागि चितवन राष्ट्रिय निकुञ्ज र बर्दिया राष्ट्रिय निकुञ्जजस्ता क्षेत्रहरूले योगदान पुर्‍याएका छन्। यी ठाउँहरूमा गैंडा, बाघ, हात्तीजस्ता दुर्लभ जनावरहरू भेटिन्छन्।

तर, नेपालका केही चुनौतीहरू पनि छन्। राजनीतिक अस्थिरता, गरिबी, र बेरोजगारीले देशको प्रगतिमा अवरोध ल्याएको छ। धेरै नेपाली युवा रोजगारीको खोजीमा विदेश जाने गरेका छन्। यद्यपि, सरकार र विभिन्न सामाजिक संस्थाहरूले यी समस्याहरू समाधान गर्न प्रयासरत छन्। नेपालको पर्यटन उद्योगलाई अझ सुदृढ बनाउन र कृषिलाई आधुनिकीकरण गर्नका लागि योजनाहरू बनाइँदैछन्।

नेपालको प्राकृतिक र सांस्कृतिक सम्पदालाई विश्वभर प्रचार-प्रसार गर्नु आवश्यक छ। विश्वका मानिसहरूले नेपालको वास्तविक सुन्दरता र विविधता बुझून् भन्ने नेपाल सरकार र पर्यटन क्षेत्रका लागि ठूलो चुनौती र अवसर हो।

Summarize the text in around 250 words in nepali.
"""
sys_mon = SystemMonitor()
sys_mon.start_monitoring()

response = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=input_data)
    + "<start_of_turn>model\n",
    device=device,
    output_len=500,
)

sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [34]:
print(response)

नेपाल, विश्वको १४ अग्ला हिमालि मध्ये ८ हिमालको घर, हो। यस देशलाई विश्वदृष्टिमा विशेष बनाउँदा ऋषि धरोहर, विविध संस्कृति, र जगतगर्को पर्यटन बाजारको मनमोहक स्थल समेटिएको छ। यसमा नेवार, मगर, थारु, लिम्बु, शेर्पाजस्ता जातिहरू समेतको बसोबास छ।  


प्राकृतिक सुन्दरीको साथै धार्मिक महत्व पनि नेपालमा देखिन्छ। लुम्बिनी बुद्धको जन्मस्थल, पशुपतिनाथ मन्दिर, चाङ्गुनारायण, स्वयम्भूनाथ जस्ता महत्त्वपूर्ण धार्मिक स्थल नेपालमा मुख्य आकर्षणको रूपमा बनेको छ। 

नेपालले साहसिक पर्यटकहरूको आकर्षणको आधार जस्तै अन्नपूर्ण बेस क्याम्प, लाङटाङ उपत्यका, रारा ताल, फेवा ताल जस्ता गन्तव्यहरूमा आकर्षण दर्श्न मद्दत गर्छ। वन्यजन्तु संरक्षणका लागि चितवन र बर्दिया राष्ट्रिय निकुञ्ज भर्ने योगदानको लागि नेपालले प्रमाणित रूपमा नवाचार गरेको छ। 

तर, नेपालमा राजनीतिक अस्थिरता, गरिबी, र बेरोजगारीले चुनौतीहरू पनि पर्छ। यसले देशको प्रगतिको अवरोध पर्‍याचित छ। हालसम्म नेपालको युवाहरू कमजोर धेरै विदेश आउने र रोजगारीको खोजी गर्ने।  सरकार र सामाजिक संस्थाहरूले समाधानका लागि सुरुआत गरिएको छ भने नेपालका पर्यटन उद्योगको विकास र कृषि आधुन

In [35]:
plot_system_usage(sys_mon.get_data())

## Translation Test

In [36]:
input_data = """
नेपाल दक्षिण एशियाको एक सानो, तर विविधताले भरिएको देश हो। यो देश विश्वका १४ अग्ला हिमालमध्ये ८ वटा हिमालको घर हो, जसमा सगरमाथा (माउन्ट एभरेस्ट) समेत समावेश छ। नेपालको प्राकृतिक सौन्दर्यले मात्र होइन, सांस्कृतिक र ऐतिहासिक महत्त्वले पनि यसलाई विशेष बनाएको छ। यहाँ विभिन्न जातजाति, भाषा, धर्म र संस्कृति समेटिएका छन्। नेपालमा नेवार, मगर, थारु, लिम्बु, शेर्पाजस्ता जातिहरूको बसोबास छ, जसले नेपाली समाजलाई विविधताले भरिपूर्ण बनाएको छ।

धार्मिक दृष्टिकोणले, नेपाल हिन्दू र बौद्ध धर्मका लागि एक पवित्र स्थल मानिन्छ। लुम्बिनी बुद्धको जन्मस्थल हो, जहाँ विश्वभरका पर्यटकहरू दर्शनका लागि आउने गर्छन्। पशुपतिनाथ मन्दिर, चाङ्गुनारायण, स्वयम्भूनाथ, बौद्धनाथ जस्ता धार्मिक स्थलहरू पनि नेपालका मुख्य आकर्षण हुन्। नेपालको राजधानी काठमाडौं तीन हजार वर्ष पुरानो इतिहास बोकेको शहर हो। यहाँ मल्ल राजाहरूले निर्माण गरेका दरबार, मन्दिर र कलात्मक सम्पदा भेट्न सकिन्छ।

नेपालको प्राकृतिक सौन्दर्यले साहसिक पर्यटकहरूलाई आकर्षित गर्छ। अन्नपूर्ण बेस क्याम्प, लाङटाङ उपत्यका, रारा ताल, फेवा तालजस्ता गन्तव्यहरूमा हाइकिङ, ट्रेकिङ, र क्याम्पिङ गर्न पर्यटकहरू आउने गर्छन्। वन्यजन्तु संरक्षणका लागि चितवन राष्ट्रिय निकुञ्ज र बर्दिया राष्ट्रिय निकुञ्जजस्ता क्षेत्रहरूले योगदान पुर्‍याएका छन्। यी ठाउँहरूमा गैंडा, बाघ, हात्तीजस्ता दुर्लभ जनावरहरू भेटिन्छन्।

तर, नेपालका केही चुनौतीहरू पनि छन्। राजनीतिक अस्थिरता, गरिबी, र बेरोजगारीले देशको प्रगतिमा अवरोध ल्याएको छ। धेरै नेपाली युवा रोजगारीको खोजीमा विदेश जाने गरेका छन्। यद्यपि, सरकार र विभिन्न सामाजिक संस्थाहरूले यी समस्याहरू समाधान गर्न प्रयासरत छन्। नेपालको पर्यटन उद्योगलाई अझ सुदृढ बनाउन र कृषिलाई आधुनिकीकरण गर्नका लागि योजनाहरू बनाइँदैछन्।

नेपालको प्राकृतिक र सांस्कृतिक सम्पदालाई विश्वभर प्रचार-प्रसार गर्नु आवश्यक छ। विश्वका मानिसहरूले नेपालको वास्तविक सुन्दरता र विविधता बुझून् भन्ने नेपाल सरकार र पर्यटन क्षेत्रका लागि ठूलो चुनौती र अवसर हो।

Translate the text to english.
"""
sys_mon = SystemMonitor()
sys_mon.start_monitoring()

response = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=input_data)
    + "<start_of_turn>model\n",
    device=device,
    output_len=500,
)

sys_mon.stop_monitoring()

Monitoring started.
Monitoring stopped.


In [37]:
print(response)

Nepal is a small but diverse country in South Asia. It is home to eight of the world's Fourteen peaks of Mount Everest, including Mount Everest.  Nepal's natural beauty is not only limited to its landscapes but also its cultural and historical significance. This country is a conglomeration of diverse communities, languages, religions and cultures.  Various ethnic groups such as the Newar, Magar, Tharu, Limbu, and Sherpa live in Nepal, adding to the rich tapestry of Nepal's society.

From a religious perspective, Nepal is a sacred place for Hinduism and Buddhism. Lumbini is considered the birthplace of Buddha, which attracts Buddhists and tourists from around the world.  Pashupati Nath temple, Changunarayan, Swayambhunath (Monkey Temple), and Buddhist temples like Boudhathnath are also significant attractions in Nepal. Kathmandu, the capital of Nepal, boasts a rich history spanning 3,000 years, featuring ancient palaces, temples, and artistic landmarks.


Nepal's natural beauty attracts

In [38]:
plot_system_usage(sys_mon.get_data())

## Output Length VS time

In [18]:
input_data = "What is capital of Kathmandu"

output_len = [100,200,300,400,500,600,700,800]
time_elapsed = []
for length in output_len:
    start_time = time.time()
    response = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=input_data)
        + "<start_of_turn>model\n",
        device=device,
        output_len=length,
    )
    time_elapsed.append(time.time()-start_time)


In [19]:
fig = go.Figure()

# Add a line plot
fig.add_trace(go.Scatter(x=output_len, y=time_elapsed, mode='lines', name='Line Plot'))


# Add title and labels
fig.update_layout(
    title='Output Length Vs Time Elapsed',
    xaxis_title='Output Length',
    yaxis_title='Time Elapsed'
)

# Show the plot
fig.show()